# Doctor Semmelweis discovery

In [ ]:
import pandas as pd
import matplotlib as plt
import seaborn as sns

In [1]:
plt.rcParams['figure.figsize'] = [10, 8]  # for size


In [2]:
df_1 = pd.read_csv("monthly_deaths.csv")
df_2 = pd.read_csv("yearly_deaths_by_clinic.csv")
print("df_1:", df_1.shape, "df_2: ", df_2.shape)

df_1: (98, 3) df_2:  (12, 4)


In [5]:
df_1["proportion_deaths"] = df_1["deaths"]/df_1["births"]
df_2["proportion_deaths"] = df_2["deaths"]/df_2["births"]

In [6]:
df_1.head()

,date,births,deaths,proportion_deaths
0,1841-01-01,254,37,0.145669
1,1841-02-01,239,18,0.075314
2,1841-03-01,277,12,0.043321
3,1841-04-01,255,4,0.015686
4,1841-05-01,255,2,0.007843


In [7]:
df_2.head()

,year,births,deaths,clinic,proportion_deaths
0,1841,3036,237,clinic 1,0.078063
1,1842,3287,518,clinic 1,0.157591
2,1843,3060,274,clinic 1,0.089542
3,1844,3157,260,clinic 1,0.082357
4,1845,3492,241,clinic 1,0.069015
